# MSc DS: Learning a full convariance Gaussian with missing data

In [1]:
import tensorflow as tf
import numpy as np
import scipy.stats
import scipy.io
import scipy.sparse
from scipy.io import loadmat
import pandas as pd
import tensorflow_probability as tfp
tfd = tfp.distributions
tfk = tf.keras
tfkl = tf.keras.layers
from PIL import Image
import matplotlib.pyplot as plt

We load the Iris data set.

In [2]:
from sklearn.datasets import load_iris
data = load_iris(True)[0]

We now standardise the data:

In [3]:
xfull = ((data - np.mean(data,0))/np.std(data,0)).astype(np.float32)
n = xfull.shape[0] # number of observations
p = xfull.shape[1] # number of features

We remove 30% of the values

In [4]:
perc_miss = 0.3 # 30% of missing data
xmiss = np.copy(xfull)
xmiss_flat = xmiss.flatten()
miss_pattern = np.random.choice(n*p, np.floor(n*p*perc_miss).astype(np.int), replace=False)
xmiss_flat[miss_pattern] = np.nan 
xmiss = xmiss_flat.reshape([n,p]) # in xmiss, the missing values are represented by nans
mask = np.isfinite(xmiss) # binary mask that indicates which values are missing

We want to learn a Gaussian distribution:
$$p(x) = \mathcal{N}(x|\mu,\Sigma), $$
where $\Sigma$ is a not a diagonal matrix, using maximum likelihood. We'll use the Cholesky decomposition of $\Sigma$.

In [5]:
dim_triangular_matrix = int(p*(p+1)/2)

mu = tf.Variable(tf.random.normal([p]), dtype=tf.float32)
C = tf.Variable(tfp.math.fill_triangular(tf.random.normal([dim_triangular_matrix])), dtype=tf.float32) # log-sd of the Gaussian

We first use define a function that can compute the likelihood of a complete data point.

In [6]:
@tf.function
def log_likelihood(x):
  L = C
  L = tf.linalg.set_diag(L,tf.exp(tf.linalg.diag_part(C)))
  Sigma = tf.matmul(L,L, transpose_b=True)
  p_x = tfd.MultivariateNormalFullCovariance(loc = mu, covariance_matrix= Sigma)
  return(p_x.log_prob(x))

Now, I want you to write a function that computes the likelihood of an incomplete data point $\log p(x^{obs})$.

To help you, I'm showing you how to extract the relevant parts of $\mu$ and $\Sigma$ (which is a be tedious in TF), for a given mask $m \in \{0,1\}$:

In [7]:
x = xmiss[2,:]
m = mask[2,:]

In [8]:
mu_m = mu[m]

In [9]:
L = C
L = tf.linalg.set_diag(L,tf.exp(tf.linalg.diag_part(C)))
Sigma = tf.matmul(L,L, transpose_b=True)
q = tf.shape(tf.where(m))[0] #number of missing values
bin_mat = tf.reshape(tf.cast(m,tf.int32),[p,1]) * tf.reshape(tf.cast(m,tf.int32),[1,p]) #binary matrix
Sigma_m = tf.reshape(tf.gather_nd(Sigma, tf.where(bin_mat)),[q,q])

Now we perform SGD!

In [ ]:
params = [mu] + [C]

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
def train_step(data,mask):
  with tf.GradientTape() as tape: # the gradient tape saves all the step that needs to be saved fopr automatic differentiation
    loss = -log_likelihood_incomplete(data,mask)  # the loss is the average negative log likelihood
  gradients = tape.gradient(loss, params)  # here, the gradient is automatically computed
  optimizer.apply_gradients(zip(gradients, params))  # Adam iteration

In [ ]:
train_data_incomplete = tf.data.Dataset.from_tensor_slices((xmiss,mask)).shuffle(p).batch(1) 

In [ ]:
EPOCHS = 601

for epoch in range(1,EPOCHS+1):
  for data,m in train_data_incomplete:
    train_step(data,m) # Adam iteration
  if (epoch % 100) == 1:
    ll_on_complete_data = tf.reduce_mean(log_likelihood(xfull))
    print('Epoch  %g' %epoch)
    print('Training log-likelihood %g' %ll_on_complete_data.numpy())
    print('-----------')

Epoch  1
Training log-likelihood -6.69685
-----------
Epoch  101
Training log-likelihood -3.40287
-----------
Epoch  201
Training log-likelihood -3.40422
-----------
Epoch  301
Training log-likelihood -3.41622
-----------


KeyboardInterrupt: ignored

In [ ]:
  L = C
  L = tf.linalg.set_diag(L,tf.exp(tf.linalg.diag_part(C)))
  Sigma = tf.matmul(L,L, transpose_b=True)
  print(Sigma)

tf.Tensor(
[[ 0.9862426  -0.04387227  0.88385856  0.7918092 ]
 [-0.04387227  1.0491437  -0.35085934 -0.31857923]
 [ 0.88385856 -0.35085934  1.0254388   0.96194303]
 [ 0.7918092  -0.31857923  0.96194303  0.9548055 ]], shape=(4, 4), dtype=float32)


In [ ]:
 np.cov(xfull, rowvar=False)

array([[ 1.00671141, -0.11835884,  0.87760447,  0.82343068],
       [-0.11835884,  1.0067114 , -0.43131554, -0.36858316],
       [ 0.87760447, -0.43131554,  1.0067114 ,  0.96932763],
       [ 0.82343068, -0.36858316,  0.96932763,  1.00671144]])